# VAE

In [1]:
import sys
import os
sys.path.append( '..' )
from VAE.vae import *
from VAE.VAE_smac import *
from helpers.pc_stats import *
import mlflow

## Parameter definition

In [2]:
data_dir  = os.path.normpath(os.path.join(os.getcwd(), "../../runs/FIA/Com8_grown_together/oms"))
run_dir = os.path.normpath(os.path.join(os.getcwd(), "../../runs/VAE/hyperparameter_optimization"))
results_dir = os.path.normpath(os.path.join(os.getcwd(), "../../runs/VAE/results"))
test_configuration = False
overwrite = False
verbosity = 1
framework = "pytorch_jupyter"
outdir = Path(os.path.normpath(os.path.join(run_dir, f"smac_vae_{framework}")))

# Logging (time and steps)
last_timestamp = time.time()
step = 0
runtimes = {}


time_step(message="Setup loaded", verbosity=verbosity)

Setup loaded (0.6456279754638672s)


In [3]:
X = read_data(data_dir, verbosity=verbosity)

configuration_space = ConfigurationSpace(name="LD", seed=42)
hyperparameters = [
    Constant(       "original_dim",             X.shape[1]),
    Float(          "input_dropout",            (0.0, 0.5), default=0.25),
    Integer(        "intermediate_layers",      (1, 5), default=2),
    Integer(        "intermediate_dimension",   (1, 20), log=True, default=4),
    Categorical(    "intermediate_activation",  ["relu", "selu", "tanh", "leakyrelu"], default="leakyrelu"),
    Integer(        "latent_dimension",         (1, 10), log=False, default=2),
    Categorical(    "solver",                   ["nadam"], default="nadam"),
    Float(          "learning_rate",            (1e-4, 1e-2), log=True, default=1e-3)
]
configuration_space.add_hyperparameters(hyperparameters)
forbidden_clauses = [
    ForbiddenGreaterThanRelation(configuration_space["latent_dimension"], configuration_space["intermediate_dimension"])
]
configuration_space.add_forbidden_clauses(forbidden_clauses)
if verbosity > 0: 
    print(f"Configuration space defined with estimated {configuration_space.estimate_size()} possible combinations.\n")

Data loaded (5.064542531967163s)
Configuration space defined with estimated inf possible combinations.



## Optimization definition

In [7]:
class FIA_VAE_tune:
    """
    Class for running the SMAC3 tuning
    """
    def __init__(self, X, test_size:float, configuration_space:ConfigurationSpace, model_builder,
                 log_dir:str, batch_size:int=16, verbosity:int=0, gpu:bool=False, name:str="smac_vae"):
        self.configuration_space = configuration_space
        self.model_builder = model_builder
        self.training_data, self.test_data = train_test_split(X, test_size=test_size)
        self.batch_size = batch_size
        self.log_dir = log_dir
        self.verbosity = verbosity
        self.gpu = gpu
        self.name = name
        self.count = 0

    def train(self, config:Configuration, seed:int=0, budget:int=25) -> float:
        """
        Method to train the model

        Args:
            config: Configuration to be trained upon
            seed: initializing seed
            budget: number of epochs to be used in training
        
        Returns:
            Average loss of the model
        """
        time_step("Start", verbosity=self.verbosity, min_verbosity=2)
        keras.utils.set_random_seed(seed)

        # Definition
        model = self.model_builder(config)
        if self.verbosity >= 3:
            model.summary()
            print_utilization(gpu=self.gpu)
        time_step("Model built", verbosity=self.verbosity, min_verbosity=2)

        # Fitting
        callbacks = []
        run =  mlflow.start_run(run_name=f"fia_vae_hptune_{self.count}", nested=True)
        callbacks.append( mlflow.keras.MlflowCallback(run) )
        model.fit(x=self.training_data, y=self.training_data, validation_split=0.2,
                  batch_size=self.batch_size, epochs=int(budget),
                  callbacks=callbacks, verbose=self.verbosity)

        if self.verbosity >= 3:
            print("After training utilization:")
            print_utilization(gpu=self.gpu)
        time_step("Model trained", verbosity=self.verbosity, min_verbosity=2)

        # Evaluation
        loss, recon_loss, kl_loss = model.evaluate(self.test_data, self.test_data,
                                                   batch_size=self.batch_size, verbose=self.verbosity)
        
        mlflow.log_params(config)
        mlflow.log_metrics({"eval-loss": loss, "eval-reconstruction_loss": recon_loss, "eval-kl_loss": kl_loss},
                           step=int(budget) + 1)
        time_step("Model evaluated", verbosity=self.verbosity, min_verbosity=2)        
        
        # Clearing model parameters
        keras.backend.clear_session()
        self.count += 1
        time_step("Session cleared", verbosity=self.verbosity, min_verbosity=2)
                
        return loss

In [8]:
fia_vae_tune = FIA_VAE_tune( X, test_size=0.2, configuration_space=configuration_space, model_builder=FIA_VAE,
                                batch_size=64, log_dir=os.path.join(outdir, "log"), verbosity=verbosity, gpu=False )

In [9]:
scenario = Scenario( fia_vae_tune.configuration_space, deterministic=True,
                     n_trials=20, min_budget=2, max_budget=100,
                     n_workers=1, output_directory=outdir,
                     walltime_limit=np.inf, cputime_limit=np.inf, trial_memory_limit=None )   # Max RAM in Bytes (not MB)
                    
initial_design = MultiFidelityFacade.get_initial_design(scenario, n_configs=10)
intensifier = Hyperband(scenario, incumbent_selection="highest_budget")
facade = MultiFidelityFacade( scenario, fia_vae_tune.train, 
                              initial_design=initial_design, intensifier=intensifier,
                              overwrite=overwrite, logging_level=30-verbosity*10 )
time_step(message=f"SMAC defined. Overwriting: {overwrite}", verbosity=verbosity)

[INFO][abstract_initial_design.py:82] Using `n_configs` and ignoring `n_configs_per_hyperparameter`.
[INFO][abstract_initial_design.py:95] Reducing the number of initial configurations from 10 to 5 (max_ratio == 0.25).
[INFO][abstract_initial_design.py:147] Using 5 initial design configurations and 0 additional configurations.
SMAC defined. Overwriting: False (601.848438501358s)


### Search

In [10]:
with mlflow.start_run(run_name=f"fia_vae_hptune_test"):
    incumbent = run_optimization(facade=facade, smac_model=fia_vae_tune, verbose_steps=10, verbosity=3)

Starting search:


  0%|          | 0/10 [00:00<?, ?it/s]

[INFO][abstract_intensifier.py:305] Using only one seed for deterministic scenario.
[INFO][successive_halving.py:164] Successive Halving uses budget type BUDGETS with eta 3, min budget 2, and max budget 100.
[INFO][successive_halving.py:323] Number of configs in stage:
[INFO][successive_halving.py:325] --- Bracket 0: [27, 9, 3, 1]
[INFO][successive_halving.py:325] --- Bracket 1: [12, 4, 1]
[INFO][successive_halving.py:325] --- Bracket 2: [6, 2]
[INFO][successive_halving.py:325] --- Bracket 3: [4]
[INFO][successive_halving.py:327] Budgets in stage:
[INFO][successive_halving.py:329] --- Bracket 0: [3.7037037037037033, 11.11111111111111, 33.33333333333333, 100.0]
[INFO][successive_halving.py:329] --- Bracket 1: [11.11111111111111, 33.33333333333333, 100.0]
[INFO][successive_halving.py:329] --- Bracket 2: [33.33333333333333, 100.0]
[INFO][successive_halving.py:329] --- Bracket 3: [100.0]
Configuration: {'input_dropout': 0.27244159149844843, 'intermediate_activation': 'leakyrelu', 'intermed

Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step - kl_loss: -0.0000e+00 - loss: 0.0014 - reconstruction_loss: 0.0014 - val_kl_loss: 0.0013 - val_loss: 0.0016 - val_reconstruction_loss: 2.1853e-04
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 481ms/step - kl_loss: 0.0053 - loss: 0.0055 - reconstruction_loss: 1.7508e-04 - val_kl_loss: 1.9976e-04 - val_loss: 2.3407e-04 - val_reconstruction_loss: 3.4304e-05
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step - kl_loss: 7.7525e-04 - loss: 8.1374e-04 - reconstruction_loss: 3.8495e-05 - val_kl_loss: 1.2806e-04 - val_loss: 1.3254e-04 - val_reconstruction_loss: 4.4745e-06
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - kl_loss: 1.6230e-04 - loss: 1.7224e-04 - reconstruction_loss: 9.9341e-06


 10%|█         | 1/10 [00:27<04:10, 27.81s/it]

Configuration: {'input_dropout': 0.30846699843737846, 'intermediate_activation': 'selu', 'intermediate_dimension': 6, 'intermediate_layers': 2, 'latent_dimension': 2, 'learning_rate': 0.00032105778495503974, 'original_dim': 825000, 'solver': 'nadam'}


Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step - kl_loss: 4.7684e-07 - loss: 0.0010 - reconstruction_loss: 0.0010 - val_kl_loss: 1.5759e-04 - val_loss: 7.4161e-04 - val_reconstruction_loss: 5.8402e-04
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step - kl_loss: 6.1980e-04 - loss: 0.0012 - reconstruction_loss: 6.0224e-04 - val_kl_loss: -0.0000e+00 - val_loss: 3.8837e-04 - val_reconstruction_loss: 3.8837e-04
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step - kl_loss: -0.0000e+00 - loss: 3.1588e-04 - reconstruction_loss: 3.1588e-04 - val_kl_loss: 3.1888e-06 - val_loss: 1.9292e-04 - val_reconstruction_loss: 1.8973e-04
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - kl_loss: 4.2021e-06 - loss: 9.2426e-05 - reconstruction_loss: 8.8224e-05


 20%|██        | 2/10 [00:55<03:41, 27.65s/it]

Configuration: {'input_dropout': 0.0690914756743069, 'intermediate_activation': 'leakyrelu', 'intermediate_dimension': 1, 'intermediate_layers': 5, 'latent_dimension': 1, 'learning_rate': 0.0003910933172251061, 'original_dim': 825000, 'solver': 'nadam'}


In [9]:
from keras import backend

print(backend.backend())

torch
